# **Volcengine Agent Development Kit 教程**

## 介绍

VeADK 是由火山引擎推出的一套面向智能体开发的全流程框架，旨在为开发者提供面向智能体构建、云端部署、评测与优化的全流程开发。

相较于现有的智能体开发框架，VeADK 具备与火山引擎产品体系深度融合的优势，帮助开发者更高效地构建企业级 AI 智能体应用。

## 配置

### 安装 VeADK

In [ ]:
%pip install veadk-python --quiet

> 如果你在本地使用的终端是`zsh`，当你进行`pip install`时，**依赖包名称应当被双引号包裹起来**，否则会解析错误。
>
> 你应当使用：
> ```bash
> pip install "veadk-python"
> pip install "agent-pilot-sdk>=0.0.9"
> ```

### 环境变量

In [ ]:
import os

os.environ["MODEL_AGENT_NAME"] = "doubao-seed-1-6-250615"  # <-- 火山方舟大模型名称

os.environ["MODEL_AGENT_API_KEY"] = ""  # <-- 设置火山方舟的API KEY来访问模型

os.environ["LOGGING_LEVEL"] = "ERROR"  # <-- 调整日志等级

## Agent 构建

### 快速开始

初始化Agent过程中，你可以不传入任何参数，VeADK 会自动读取环境变量。查看[完整的 Agent 参数](https://volcengine.github.io/veadk-python/agent/agent#%E9%80%89%E9%A1%B9)。生产环境下**使用`Runner`来运行 Agent**，通过`app_name`、`user_id`以及`session_id`以实现多租户隔离。

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

agent = Agent()
short_term_memory = ShortTermMemory()  # 短期记忆代表的是用户单次会话内的对话记录

runner = Runner(
    agent=agent, short_term_memory=short_term_memory, app_name=app_name, user_id=user_id
)

# 使用 runner 中的`run`方法来调用 Agent
response = await runner.run(
    messages="你好，我是一名 Agent 开发者。", session_id=session_id
)

print(response)

Hello! It's a pleasure to connect with an Agent developer—your work is at the cutting edge of intelligent automation and problem-solving. As veAgent, developed by the VeADK team, I’m familiar with the nuances of building capable agents, so I’d love to learn more about your projects!  

Are you focusing on a specific domain for your Agent (e.g., data science workflows, task automation, customer service, or niche applications like healthcare/finance)? Or perhaps you’re exploring technical challenges—like improving reasoning capabilities, integrating tools (APIs, databases, code execution), or optimizing user interaction flows?  

Feel free to share details about your current work, tools you’re using (e.g., frameworks like LangChain, AutoGPT, or custom architectures; languages like Python, JavaScript), or even pain points you’re troubleshooting. I’m happy to brainstorm solutions, discuss best practices, or dive into technical details with you! 😊


### 使用 Tools

初始化 Agent 时，你可以使用`tools`字段来挂载工具。VeADK 中集成了多个[内置工具](https://volcengine.github.io/veadk-python/tools/builtin-tools)，如下以一个查询天气的 demo 工具为例：

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory
from veadk.tools.demo_tools import get_city_weather

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

agent = Agent(tools=[get_city_weather])
short_term_memory = ShortTermMemory()

runner = Runner(
    agent=agent, short_term_memory=short_term_memory, app_name=app_name, user_id=user_id
)

response = await runner.run(messages="北京的天气怎么样？", session_id=session_id)

print(response)

北京现在天气晴朗，气温25°C。


你还可以体验**火山引擎提供的强大搜索能力**，例如使用`web_search`和`VeSearch`工具等。

使用[`web_search`工具](https://www.volcengine.com/docs/85508/1650263)，使用前需提前填写火山引擎的 AK 和 SK：

In [ ]:
import os

# 设置火山引擎 AK 和 SK 来使用 web_search 工具
os.environ["VOLCENGINE_ACCESS_KEY"] = ""
os.environ["VOLCENGINE_SECRET_KEY"] = ""

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory
from veadk.tools.builtin_tools.web_search import web_search

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

agent = Agent(tools=[web_search])
short_term_memory = ShortTermMemory()

runner = Runner(
    agent=agent, short_term_memory=short_term_memory, app_name=app_name, user_id=user_id
)

response = await runner.run(messages="北京今天的空气质量", session_id=session_id)

print(response)

北京今天（10月21日）的空气质量为**优**，具体信息如下：  
- **实时空气质量指数（AQI）**：19，污染级别1级，首要污染物为“--”（无显著污染物）。  
- **主要污染物浓度**：  
  - PM2.5：12 μg/m³  
  - PM10：15 μg/m³  
  - O₃：20 μg/m³  
  - SO₂：1 μg/m³  
  - NO₂：4 μg/m³  
  - CO：3 mg/m³  
- **健康影响与建议**：空气质量令人满意，基本无空气污染，各类人群可正常活动。  

（数据来源：实时空气质量监测及中国气象局信息）


尝试使用[`VeSearch`工具](https://www.volcengine.com/docs/85508/1512748)，使用前需进行[工具创建](https://console.volcengine.com/ask-echo/my-agent)，并填写如下 `VeSearch` 工具的 ENDPOINT 和 API KEY：

In [ ]:
import os

# 设置 vesearch 工具的 ENDPOINT 和 API KEY
os.environ["TOOL_VESEARCH_ENDPOINT"] = ""
os.environ["TOOL_VESEARCH_API_KEY"] = ""

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory
from veadk.tools.builtin_tools.vesearch import vesearch

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

agent = Agent(tools=[vesearch])
short_term_memory = ShortTermMemory()

runner = Runner(
    agent=agent, short_term_memory=short_term_memory, app_name=app_name, user_id=user_id
)

response = await runner.run(messages="未来七天北京的天气", session_id=session_id)

print(response)

### 北京未来七天天气预报（2025年10月21日-10月27日）  
#### **10月21日（周二）**  
- **天气**：晴转多云  
- **气温**：2℃~13℃  
- **风向风力**：北风1级  
- **提示**：昼夜温差大，早晚需穿厚外套，空气质量优（AQI 19）。  

#### **10月22日（周三）**  
- **天气**：多云转小雨  
- **气温**：4℃~10℃  
- **风向风力**：北风1级  
- **提示**：夜间有零星小雨，外出需携带雨具，洗车指数不宜。  

#### **10月23日（周四）**  
- **天气**：多云  
- **气温**：3℃~12℃  
- **风向风力**：北风1级  
- **提示**：湿度上升至62%，早晚体感较凉，建议佩戴围巾。  

#### **10月24日（周五）**  
- **天气**：多云转阴  
- **气温**：6℃~14℃  
- **风向风力**：西南风1级  
- **提示**：气温小幅回升，空气质量良，适宜室内通风。  

#### **10月25日（周六）**  
- **天气**：晴  
- **气温**：5℃~13℃  
- **风向风力**：西南风1级  
- **提示**：晴天为主，紫外线中等，户外活动建议涂防晒霜。  

#### **10月26日（周日）**  
- **天气**：晴  
- **气温**：3℃~13℃  
- **风向风力**：西北风1级  
- **提示**：湿度降至38%，空气干燥，需注意补水。  

#### **10月27日（周一）**  
- **天气**：晴  
- **气温**：3℃~12℃  
- **风向风力**：西南风1级  
- **提示**：气温稳定，适宜出行，但早间最低温仅3℃，需注意头部保暖。  

### **综合建议**  
1. **保暖**：夜间气温普遍低于5℃，建议携带羽绒服或厚大衣。  
2. **降水**：22日夜间小雨短暂影响，避免夜间长时间户外活动。  
3. **健康**：空气干燥，注意呼吸道保湿，可使用加湿器。  

（数据来源：北京市气象部门实时监测）


### 短期记忆

你可以初始化一个`Local`模式的短期记忆，它只会存在与内存中。其仅在一次调用中会有记忆。

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

agent = Agent()
short_term_memory = ShortTermMemory(
    backend="local"
)  # 指定 local 后端，或直接 ShortTermMemory()

runner = Runner(
    agent=agent, short_term_memory=short_term_memory, app_name=app_name, user_id=user_id
)

response = await runner.run(
    messages="我叫VeADK", session_id=session_id
)

print(response)

response = await runner.run(
    messages="你还记得我叫什么吗？", session_id=session_id
)

print(response)

Hello VeADK! It's wonderful to meet you. I'm veAgent, the AI agent developed by the VeADK team. Whether you need help with data science tasks, documentation writing, coding and programming, or using tools to accomplish specific goals, feel free to let me know—I’m here to assist! 😊
Of course I remember! Your name is VeADK. It's nice to connect with you again—feel free to let me know what I can help with today! 😊


VeADK 还支持你将短期记忆持久化存储在云端，未来的某一时刻你可以加载历史对话。

使用 [MySQL](https://www.volcengine.com/docs/6313) 作为短期记忆的数据库后端，使用前[创建 MySQL](https://www.volcengine.com/product/rds-mysql) 并填写如下信息：

In [ ]:
import os

os.environ["DATABASE_MYSQL_HOST"] = ""
os.environ["DATABASE_MYSQL_USER"] = ""
os.environ["DATABASE_MYSQL_PASSWORD"] = ""
os.environ["DATABASE_MYSQL_DATABASE"] = ""
os.environ["DATABASE_MYSQL_CHARSET"] = "utf8"

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

agent = Agent()
short_term_memory = ShortTermMemory(backend="mysql")  # 指定 mysql 后端
runner = Runner(
    agent=agent, short_term_memory=short_term_memory, app_name=app_name, user_id=user_id
)

prompt = "我在 7 月 15 日购买了 20 个冰激凌"
response = await runner.run(messages=prompt, session_id=session_id)
print(response)

prompt = "我什么时候买了冰激凌？"
response = await runner.run(messages=prompt, session_id=session_id)
print(response)

You bought 20 ice creams on July 15th—what a delightful purchase! Was there a special occasion (like a gathering with friends or family) that prompted buying so many, or did you just want to have a variety of sweet treats on hand to enjoy over time? 😊
你在7月15日购买了冰激凌。


### 长期记忆

如果您使用知识库、长期记忆等进阶功能，请进一步安装 veadk-python 中的扩展包：

In [ ]:
%pip install veadk-python[extensions] --quiet

使用 [OpenSearch](https://www.volcengine.com/docs/6465/?lang=zh) 作为长期记忆的数据库后端，使用前[创建 OpenSearch](https://www.volcengine.com/product/es) 并填写如下信息：

In [ ]:
import os

# OpenSearch 配置
os.environ["DATABASE_OPENSEARCH_HOST"] = ""
os.environ["DATABASE_OPENSEARCH_PORT"] = ""
os.environ["DATABASE_OPENSEARCH_USERNAME"] = ""
os.environ["DATABASE_OPENSEARCH_PASSWORD"] = ""

# Embedding 配置（使用 OpenSearch 时，需要对文本进行向量化处理）
# 设置访问火山方舟的 Embedding 模型
os.environ["MODEL_EMBEDDING_NAME"] = "doubao-embedding-text-240715"
os.environ["MODEL_EMBEDDING_API_BASE"] = "https://ark.cn-beijing.volces.com/api/v3/"
os.environ["MODEL_EMBEDDING_DIM"] = "2560"
os.environ["MODEL_EMBEDDING_API_KEY"] = ""

In [ ]:
from veadk import Agent, Runner
from veadk.memory.long_term_memory import LongTermMemory
from veadk.memory.short_term_memory import ShortTermMemory

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"

# 初始化一个长期记忆，采用 OpenSearch 向量化存储
# 长期记忆是跨 Session 的
long_term_memory = LongTermMemory(
    backend="opensearch", app_name=app_name
)

agent = Agent(long_term_memory=long_term_memory)

runner = Runner(
    agent=agent,
    app_name=app_name,
    user_id=user_id,
    short_term_memory=ShortTermMemory(),
)


# ===== 插入记忆 =====
session_id = "veadk_playground_session"
teaching_prompt = "我上周五购买了一支冰激凌。"

await runner.run(messages=teaching_prompt, session_id=session_id)
await runner.save_session_to_long_term_memory(
    session_id=session_id
)  # 将 teaching prompt 和智能体回答保存到长期记忆中


# ===== 检验记忆 =====
session_id = "veadk_playground_session_2"  # 使用一个新的 Session 来检测跨 Session 检索
student_prompt = "我上周五购买了什么? 用中文回答我。"

response = await runner.run(messages=student_prompt, session_id=session_id)

print(response)

你上周五购买了一支冰激凌。


使用 [Viking Memory](https://www.volcengine.com/docs/84313/1860687) 作为长期记忆的后端，使用前创建 Viking Memory 并填写如下信息：

In [ ]:
import os

# Viking 记忆库配置
os.environ["DATABASE_VIKING_PROJECT"] = ""
os.environ["DATABASE_VIKING_REGION"] = "cn-beijing"

In [ ]:
from veadk import Agent, Runner
from veadk.memory.long_term_memory import LongTermMemory
from veadk.memory.short_term_memory import ShortTermMemory

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"

# 初始化一个长期记忆，采用 Viking Memory 存储
# 长期记忆是跨 Session 的
long_term_memory = LongTermMemory(backend="viking", app_name=app_name, user_id=user_id)

agent = Agent(long_term_memory=long_term_memory)

runner = Runner(
    agent=agent,
    app_name=app_name,
    user_id=user_id,
    short_term_memory=ShortTermMemory(),
)


# ===== 插入记忆 =====
session_id = "veadk_playground_session_3"
teaching_prompt = "我这周三购买了三本书。"

await runner.run(messages=teaching_prompt, session_id=session_id)
await runner.save_session_to_long_term_memory(
    session_id=session_id
)  # 将 teaching prompt 和智能体回答保存到长期记忆中

由于初始使用 Viking Memory 需要构建索引，因此需要等待 1-2 分钟。等待过后，执行如下代码检验：

In [ ]:
# ===== 检验记忆 =====
session_id = "veadk_playground_session_4"  # 使用一个新的 Session 来检测跨 Session 检索
student_prompt = "我这周三购买了什么? 用中文回答我。"

response = await runner.run(messages=student_prompt, session_id=session_id)

print(response)

你这周三（2025年10月22日）购买了三本书。


### 知识库

如果您使用知识库、长期记忆等进阶功能，请进一步安装 veadk-python 中的扩展包：

In [ ]:
%pip install veadk-python[extensions] --quiet

使用 [OpenSearch](https://www.volcengine.com/product/es) 作为知识库的后端：

In [ ]:
import os

# OpenSearch 配置
os.environ["DATABASE_OPENSEARCH_HOST"] = ""
os.environ["DATABASE_OPENSEARCH_PORT"] = "9200"
os.environ["DATABASE_OPENSEARCH_USERNAME"] = ""
os.environ["DATABASE_OPENSEARCH_PASSWORD"] = ""

# 设置访问火山方舟的 Embedding 模型
os.environ["MODEL_EMBEDDING_NAME"] = "doubao-embedding-text-240715"
os.environ["MODEL_EMBEDDING_API_BASE"] = "https://ark.cn-beijing.volces.com/api/v3/"
os.environ["MODEL_EMBEDDING_DIM"] = "2560"
os.environ["MODEL_EMBEDDING_API_KEY"] = ""

In [ ]:
# 准备知识库文档并保存到本地
content = """
        The secret of red is red_000000.
        The secret of green is green_000111.
        The secret of blue is blue_000222.
        The secret of yellow is yellow_000333.
"""

knowledgebase_file = "/tmp/knowledgebase.md"
with open(knowledgebase_file, "w", encoding="utf-8") as f:
    f.write(content)

print(f"Knowledgebase file path: {knowledgebase_file}")

Knowledgebase file path: /tmp/knowledgebase.md


In [ ]:
from veadk import Agent, Runner
from veadk.knowledgebase.knowledgebase import KnowledgeBase
from veadk.memory.short_term_memory import ShortTermMemory

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"


knowledgebase = KnowledgeBase(
    backend="opensearch", app_name=app_name
)  # 指定 opensearch 后端
knowledgebase.add_from_files(files=[knowledgebase_file])

agent = Agent(knowledgebase=knowledgebase)

runner = Runner(
    agent=agent,
    short_term_memory=ShortTermMemory(),
    app_name=app_name,
    user_id=user_id,
)

response = await runner.run(
    messages="Tell me the secret of green.", session_id=session_id
)
print(response)

The secret of green is green_000111.


使用 [Viking DB](https://www.volcengine.com/docs/84313/1254463) 作为知识库的后端，使用前创建 Viking DB 和 [TOS](https://www.volcengine.com/docs/6349) 并填写如下信息：

In [ ]:
import os

# Viking DB 配置
os.environ["DATABASE_VIKING_PROJECT"] = ""
os.environ["DATABASE_VIKING_REGION"] = "cn-beijing"

# 存储桶配置
os.environ["DATABASE_TOS_ENDPOINT"] = "tos-cn-beijing.volces.com"
os.environ["DATABASE_TOS_REGION"] = "cn-beijing"
os.environ["DATABASE_TOS_BUCKET"] = ""

In [ ]:
import os
import requests

# 准备知识库文档并保存到本地
pdf_url = "https://arxiv.org/pdf/1706.03762"
pdf_path = "/tmp/knowledgebase.pdf"
resp = requests.get(pdf_url)
with open(pdf_path, "wb") as f:
    f.write(resp.content)

print(f"Knowledgebase file path: {pdf_path}")

Knowledgebase file path: /tmp/knowledgebase.pdf


In [ ]:
from veadk import Agent
from veadk.knowledgebase.knowledgebase import KnowledgeBase
from veadk.memory.short_term_memory import ShortTermMemory

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

knowledgebase = KnowledgeBase(backend="viking", app_name=app_name)  # 指定 viking 后端

knowledgebase.add_from_files(files=[pdf_path])  # 直接添加文档，无需手动切片

agent = Agent(knowledgebase=knowledgebase)

由于初始使用 Viking DB 需要构建索引，因此需要等待 3-4 分钟。等待过后，执行如下代码检验：

In [ ]:
from veadk import Runner

runner = Runner(
    agent=agent,
    short_term_memory=ShortTermMemory(),
    app_name=app_name,
    user_id=user_id,
)

response = await runner.run(
    messages="What is `Multi-Head Attention`?", session_id=session_id
)
print(response)

`Multi-Head Attention` is a key component in the Transformer architecture, a fundamental model in deep learning, particularly for natural language processing (NLP) tasks. It extends the standard attention mechanism by allowing the model to jointly attend to information from different representation subspaces at different positions. Here's a detailed breakdown:


### **Core Idea**  
Instead of performing a single attention computation, multi-head attention splits the query, key, and value vectors into multiple "heads" (parallel subsets), computes scaled dot-product attention for each head independently, and then concatenates the results to form the final output. This enables the model to capture diverse types of contextual relationships (e.g., syntactic, semantic, or positional) simultaneously.


### **How It Works**  
1. **Linear Projections**:  
   The input query (\(Q\)), key (\(K\)), and value (\(V\)) matrices are projected into higher-dimensional spaces using learned linear layers.

### 多 Agent 协同

你可以通过设置`sub_agents`字段来进行多 Agent 之间的本地协同。

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

short_term_memory = ShortTermMemory()

# sub agents
python_coding_agent = Agent(
    name="python_coder",
    description="擅长使用 Python 编程语言来解决问题。",
    instruction="""使用 Python 语言来解决问题。
    注意，你生成的代码第一行需要有注释，标明作者是`python-coder`。""",
)

java_coding_agent = Agent(
    name="java_coder",
    description="擅长使用 Java 编程语言来解决问题。",
    instruction="""使用 Java 语言来解决问题。
    注意，你生成的代码第一行需要有注释，标明作者是`java-coder`。""",
)

# root agent
coding_agent = Agent(
    name="coding_agent",
    description="可以调用适合的智能体来解决用户问题。",
    instruction="调用适合的智能体来解决用户问题。",
    sub_agents=[python_coding_agent, java_coding_agent],
)


runner = Runner(
    agent=coding_agent,
    short_term_memory=ShortTermMemory(),
    app_name=app_name,
    user_id=user_id,
)

response = await runner.run(
    messages="使用 Python 帮我写一段快速排序的代码。", session_id=session_id
)

print(response)

# author: python-coder

def quick_sort(arr):
    """
    快速排序算法的Python实现（原地排序版本）
    
    参数:
        arr: 需要排序的列表
    
    返回:
        排序后的列表（原地修改原列表并返回）
    """
    def partition(low, high):
        # 选择最右侧元素作为基准值
        pivot = arr[high]
        # i是小于基准值区域的右边界
        i = low - 1
        
        for j in range(low, high):
            # 如果当前元素小于等于基准值，扩展小于区域
            if arr[j] <= pivot:
                i += 1
                arr[i], arr[j] = arr[j], arr[i]
        
        # 将基准值放到最终位置（小于区域和大于区域中间）
        arr[i + 1], arr[high] = arr[high], arr[i + 1]
        return i + 1  # 返回基准值的索引
    
    def _quick_sort(low, high):
        if low < high:
            # 获取基准值位置并分区
            pivot_index = partition(low, high)
            # 递归排序左侧子数组
            _quick_sort(low, pivot_index - 1)
            # 递归排序右侧子数组
            _quick_sort(pivot_index + 1, high)
    
    # 初始调用，排序整个数组
    _quick_sort(0, len(arr) - 1)
    return arr

# 测试示例
if __name__ == "__main__":
    test_arr = [6, 3, 8

## 可观测性

### 本地观测

可以通过初始化 OpentelemetryTracer 来将 Agent 的调用栈存储为本地文件：

In [ ]:
import json

from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory
from veadk.tools.demo_tools import get_city_weather
from veadk.tracing.telemetry.opentelemetry_tracer import OpentelemetryTracer

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

tracer = OpentelemetryTracer()

agent = Agent(
    tools=[get_city_weather], tracers=[tracer]
)  # 创建一个配置 tracers 的 Agent

short_term_memory = ShortTermMemory()

runner = Runner(
    agent=agent, short_term_memory=short_term_memory, app_name=app_name, user_id=user_id
)

# 如果在 agent 中设置了tracer，
# 并且在 `run` 中指定保存，runner将会尝试保存 tracing 文件
await runner.run(
    messages="北京的天气怎么样？", session_id=session_id, save_tracing_data=True
)

print(f"Tracing file path: {tracer._trace_file_path}")

with open(tracer._trace_file_path, "r") as f:
    tracing_content = f.read()

print(f"Tracing file content:\n{json.loads(tracing_content)}")

Tracing file path: /tmp/veadk_opentelemetry_tracer_veadk_playground_user_veadk_playground_session_2410ceef03af7a13e8b08eb77d353457.json
Tracing file content:
[{'name': 'execute_tool get_city_weather', 'span_id': 17742897692489967014, 'trace_id': 47939481711043315363740261242505606231, 'start_time': 1761031433773324000, 'end_time': 1761031433773601000, 'attributes': {'gen_ai.system': 'openai', 'gen_ai.system.version': '0.2.10', 'gen_ai.agent.name': 'veAgent', 'openinference.instrumentation.veadk': '0.2.10', 'gen_ai.app.name': 'veadk_playground_app', 'gen_ai.user.id': 'veadk_playground_user', 'gen_ai.session.id': 'veadk_playground_session', 'agent_name': 'veAgent', 'agent.name': 'veAgent', 'app_name': 'veadk_playground_app', 'app.name': 'veadk_playground_app', 'user.id': 'veadk_playground_user', 'session.id': 'veadk_playground_session', 'cozeloop.report.source': 'veadk', 'cozeloop.call_type': '', 'gen_ai.operation.name': 'execute_tool', 'gen_ai.tool.name': 'get_city_weather', 'gen_ai.too

### 云端观测

需要设置上报器（exporter），VeADK 当前支持将 Tracing 数据上报至[火山引擎 TLS](https://www.volcengine.com/product/tls)、[火山引擎 APMPlus](https://www.volcengine.com/product/apmplus) 以及[火山引擎 Coze Loop](https://www.coze.cn/loop) 平台。它们分别对应不同的上报器。

初始化上报器之前需要进行环境变量设置：

In [ ]:
import os

# TLS 上报器环境变量
os.environ["OBSERVABILITY_OPENTELEMETRY_TLS_ENDPOINT"] = (
    "https://tls-cn-beijing.volces.com:4318/v1/traces"
)
os.environ["OBSERVABILITY_OPENTELEMETRY_TLS_REGION"] = "cn-beijing"
os.environ["OBSERVABILITY_OPENTELEMETRY_TLS_SERVICE_NAME"] = ""

# APMPlus 上报器环境变量
os.environ["OBSERVABILITY_OPENTELEMETRY_APMPLUS_ENDPOINT"] = (
    "http://apmplus-cn-beijing.volces.com:4317"
)
os.environ["OBSERVABILITY_OPENTELEMETRY_APMPLUS_SERVICE_NAME"] = ""
os.environ["OBSERVABILITY_OPENTELEMETRY_APMPLUS_API_KEY"] = ""

# Coze Loop 上报器环境变量
os.environ["OBSERVABILITY_OPENTELEMETRY_COZELOOP_ENDPOINT"] = (
    "https://api.coze.cn/v1/loop/opentelemetry/v1/traces"
)
os.environ["OBSERVABILITY_OPENTELEMETRY_COZELOOP_SERVICE_NAME"] = ""
os.environ["OBSERVABILITY_OPENTELEMETRY_COZELOOP_API_KEY"] = ""

初始化上报器，并装配到 Agent 中：

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory
from veadk.tools.demo_tools import get_city_weather
from veadk.tracing.telemetry.exporters.apmplus_exporter import APMPlusExporter
from veadk.tracing.telemetry.exporters.cozeloop_exporter import CozeloopExporter
from veadk.tracing.telemetry.exporters.tls_exporter import TLSExporter
from veadk.tracing.telemetry.opentelemetry_tracer import OpentelemetryTracer

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

exporters = [
    CozeloopExporter(),
    APMPlusExporter(),
    TLSExporter(),
]  # 初始化 tracing 上报器
tracer = OpentelemetryTracer(exporters=exporters)

agent = Agent(
    tools=[get_city_weather], tracers=[tracer]
)  # 创建一个配置 tracers 的 Agent

short_term_memory = ShortTermMemory()

runner = Runner(
    agent=agent, short_term_memory=short_term_memory, app_name=app_name, user_id=user_id
)

await runner.run(messages="北京的天气怎么样？", session_id=session_id, save_tracing_data=True)

print(f"Tracing file path: {tracer._trace_file_path}")

Tracing file path: /tmp/veadk_opentelemetry_tracer_veadk_playground_user_veadk_playground_session_3ae5add47d5c1c041af606c7f95221cf.json


最后，在云端平台可查看 Tracing 数据：

*   Coze Loop 平台：https://www.coze.cn/loop
    ![Coze loop 监控图](https://github.com/volcengine/veadk-python/blob/main/assets/images/tutorial-observation-cozeloop.png?raw=true)

*   APMPlus 平台：https://www.volcengine.com/product/apmplus
    ![APMPlus 监控图](https://github.com/volcengine/veadk-python/blob/main/assets/images/tutorial-observation-apmplus-tracing.png?raw=true)

*   TLS 平台：https://www.volcengine.com/product/tls




## 评测

### 运行时数据转换

可以通过 runtime recorder 将运行时的数据直接保存为 Google ADK 兼容的 Evaluation Set 文件（json 格式）：

In [ ]:
import json

from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory
from veadk.tools.demo_tools import get_city_weather

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

agent = Agent(tools=[get_city_weather])
short_term_memory = ShortTermMemory()

runner = Runner(
    agent=agent, short_term_memory=short_term_memory, app_name=app_name, user_id=user_id
)

await runner.run(messages="北京的天气怎么样？", session_id=session_id)

# 调用 runner 中的`save_eval_set`来保存评测集文件到本地
eval_set_path = await runner.save_eval_set(session_id=session_id)

print(f"Evaluation file path: {eval_set_path}")

with open(eval_set_path, "r") as f:
    data = json.load(f)
print(f"Evaluation file content\n{data}")

Evaluation file path: /tmp/veadk_playground_app/default.evalset.json
Evaluation file content
{'eval_set_id': 'default', 'name': 'default', 'description': None, 'eval_cases': [{'eval_id': 'veadk_eval_20251021090810', 'conversation': [{'invocation_id': 'e-fc239191-4057-4afd-acf3-56ad6fef9b3a', 'user_content': {'parts': [{'video_metadata': None, 'thought': None, 'inline_data': None, 'file_data': None, 'thought_signature': None, 'code_execution_result': None, 'executable_code': None, 'function_call': None, 'function_response': None, 'text': '北京的天气怎么样？'}], 'role': 'user'}, 'final_response': {'parts': [{'video_metadata': None, 'thought': None, 'inline_data': None, 'file_data': None, 'thought_signature': None, 'code_execution_result': None, 'executable_code': None, 'function_call': None, 'function_response': None, 'text': '北京现在是晴天，气温25°C。'}], 'role': 'model'}, 'intermediate_data': {'tool_uses': [], 'intermediate_responses': []}, 'creation_timestamp': 1761008876.358341}], 'session_input': {'ap

## Prompt 优化

使用火山引擎 PromptPilot 来进行 Agent 的系统提示词（System Prompt）优化。

In [ ]:
# 安装火山引擎提供的依赖
%pip install agent-pilot-sdk>=0.1.2

您可以从 Prompt Pilot 产品[官方页面](https://promptpilot.volcengine.com/)获取 KEY 和 Workspace ID，在下方设置后访问服务：

In [ ]:
import os

os.environ["PROMPT_PILOT_API_KEY"] = ""
os.environ["PROMPT_PILOT_WORKSPACE_ID"] = ""

定义一个简单的 Agent，并且让它携带天气查询的工具，体验 Prompt 优化前后的差异。

In [ ]:
from veadk import Agent
from veadk.tools.demo_tools import get_city_weather

agent = Agent(instruction="你是一个智能体", tools=[get_city_weather])

使用 Prompt Pilot 进行优化：

In [ ]:
from veadk.integrations.ve_prompt_pilot.ve_prompt_pilot import VePromptPilot

prompt_pilot = VePromptPilot(
    api_key=os.getenv("PROMPT_PILOT_API_KEY"),
    workspace_id=os.getenv("PROMPT_PILOT_WORKSPACE_ID"),
)

refined_prompt = prompt_pilot.optimize(agents=[agent])

Optimized prompt for agent veAgent:
# Role
你是veAgent，是由VeADK团队开发的AI智能体，专长于数据科学、文档处理和软件开发领域。

# Task Requirements
- **专业解答**：当用户提出与数据科学、文档处理和软件开发相关的问题时，凭借你的专业知识提供准确、详细的解答。
- **灵活运用工具**：若用户询问城市天气信息，可调用get_city_weather工具获取对应城市的天气情况，注意输入的城市名称需为英文。
- **沟通风格**：语言表达简洁明了，避免使用过于复杂的专业术语，除非用户有相关需求。在与用户交流时，保持友好、耐心的态度。

# Output Requirements
回答应紧扣用户的问题，提供有针对性的解决方案或信息。若调用工具获取信息，需确保信息准确有效，并清晰地呈现给用户。 


## 云端部署

### 部署到火山引擎 FaaS 平台

详见[部署上云](https://volcengine.github.io/veadk-python/deploy/from-scratch)。

### 端云协同

VeADK 提供了在本地直接调用/操作部署在火山引擎 FaaS 平台的 Agent 的便捷接口，方便你与云端 Agent 协作。

**以 A2A 方式调用部署的 Agent**

In [ ]:
# 部署在 VeFaaS 上的应用名称（Application name）及端点（Endpoint）
vefaas_application_name = ""
vefaas_endpoint = ""

In [ ]:
from veadk.cloud.cloud_app import CloudApp

# 设置会话属性
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

cloud_app = CloudApp(vefaas_application_name=vefaas_application_name, vefaas_endpoint=vefaas_endpoint)

# 发送消息
response_message = await cloud_app.message_send(
    message="你好，我是一名 Agent 开发者。", user_id=user_id, session_id=session_id
)

print(f"返回的消息内容: {response_message.parts[0].root.text}")

返回的消息内容: 您好呀！作为Agent开发者，或许您对如何更高效精准地使用函数获取天气数据感兴趣？要是您想测试关于天气查询的功能，不妨给我一个城市名，我调用`get_city_weather`函数来为您获取相应的天气报告，方便您做进一步研究。 


**以 MCP 方式调用部署的Agent**

使用此方式需安装 `fastmcp` 库：

In [ ]:
%pip install fastmcp --quiet

In [ ]:
# 部署在 VeFaaS 上的应用名称（Application name）及鉴权 Token
vefaas_application_name = ""
vefaas_application_token = ""

In [ ]:
from veadk.cloud.cloud_app import CloudApp
from fastmcp.client import Client

# 设置会话属性
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

cloud_app = CloudApp(vefaas_application_name=vefaas_application_name)

endpoint = cloud_app._get_vefaas_endpoint()

client = Client(f"{endpoint}/mcp?token={vefaas_application_token}")

async with client:
    # 列出所有工具
    tools = await client.list_tools()
    print(f"云部署的 MCP 工具: {tools}")

    # 通过 MCP 的方式调用云端 Agent
    res = await client.call_tool(
        "run_agent",
        {
            "user_input": "你好，我是一名 Agent 开发者。",
            "session_id": session_id,
            "user_id": user_id,
        },
    )
    print(f"返回的消息内容: {res}")

云部署的 MCP 工具: [Tool(name='run_agent', title=None, description='A reporter for weather updates\nArgs:\n    user_input: User\'s input message (required).\n    user_id: User identifier. Defaults to "mcp_user".\n    session_id: Session identifier. Defaults to "mcp_session".\nReturns:\n    Final agent response as a string.\n\n\n**Query Parameters:**\n\n- **user_input** (Required): No description.\n\n- **user_id**: No description.\n\n- **session_id**: No description.\n\n\n**Responses:**\n\n- **200** (Success): Successful Response\n  - Content-Type: `application/json`\n\n  - **Example:**\n```json\n"string"\n```\n\n- **422**: Validation Error\n  - Content-Type: `application/json`\n\n  - **Response Properties:**\n\n  - **Example:**\n```json\n{\n  "detail": [\n    "unknown_type"\n  ]\n}\n```', inputSchema={'type': 'object', 'properties': {'user_input': {'type': 'string', 'title': 'User Input'}, 'user_id': {'type': 'string', 'title': 'User Id', 'default': 'mcp_user'}, 'session_id': {'type': 'strin